# Learning to Simulate Complex Physics with Graph Neural Networks
### [*Sanchez-Gonzalez et al.*](https://github.com/google-deepmind/deepmind-research/tree/master/learning_to_simulate) *(2020)*
### Ported from TensorFlow/Sonnet + Graph Nets to TensorFlow/Keras + TensorFlow GNN.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Move into cloud storage, e.g. My Drive
%cd /content/drive/MyDrive

import os
if os.path.exists("learning_to_simulate"):
    %cd learning_to_simulate
    !git pull
else:
    !git clone https://github.com/BitTrain/learning_to_simulate.git
    %cd learning_to_simulate

## Install dependencies

In [ ]:
%pip install tensorflow_gnn

## Environment

In [1]:
import datetime, logging, os, pickle, sys

BASE_DIR = os.getcwd()
PARENT_DIR = os.path.dirname(BASE_DIR)
if PARENT_DIR not in sys.path:
    sys.path.append(PARENT_DIR)

os.environ["TF_USE_LEGACY_KERAS"] = '1'  # tensorflow_gnn requires Keras v2

import tensorflow as tf
import tensorflow_gnn as tfgnn
from learning_to_simulate import utils, settings
from learning_to_simulate.models.learned_simulator import LearnedSimulator
settings.TF_DEBUG_MODE = False  # Eager data, input checks

print("TensorFlow", tf.__version__)
print("TensorFlow GNN", tfgnn.__version__)

2025-06-09 14:04:27.354349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow 2.16.2
TensorFlow GNN 1.0.3


## Parameters

In [2]:
#  Available datasets from Google DeepMind
"""
    "WaterDrop"
    "Water"
    "Sand"
    "Goop"
    "MultiMaterial"
    "RandomFloor"
    "WaterRamps"
    "SandRamps"
    "FluidShake"
    "FluidShakeBox"
    "Continuous"
    "WaterDrop-XL"
    "Water-3D"
    "Sand-3D"
    "Goop-3D"
"""

DATASET = "WaterDrop"

params = {
    "DATASET": DATASET,
    "DATA_PATH": os.path.join(BASE_DIR, "datasets", "deepmind", DATASET),
    "MODEL_PATH": os.path.join(BASE_DIR, "datasets", "local", DATASET, "weights"),
    "OUTPUT_PATH": os.path.join(BASE_DIR, "datasets", "local", DATASET, "rollouts"),
    "MODE": "train",
    "BATCH_SIZE": 2,  # @S-G, p. 5
    "EVAL_SPLIT": "test",
    "NUM_STEPS": 20_000_000,  # tunable
    "NOISE_STD": 3e-4,  # @S-G, p. 6
    "VELOCITY_CONTEXT_SIZE": 5,  # @S-G, p. 4
    "NUM_PARTICLE_TYPES": 9,  # hardcoded
    "STATIC_PARTICLE_ID": 3,  # hardcoded
}

if not os.path.exists(params["DATA_PATH"]):
    print(f"Dataset '{DATASET}' not found at {params['DATA_PATH']}. Downloading...")
    path_to_script = os.path.join(BASE_DIR, "download_dataset.sh")
    os.system(f"bash {path_to_script} {DATASET} {os.path.dirname(params['DATA_PATH'])}")

if not os.path.exists(params["MODEL_PATH"]):
    os.makedirs(params["MODEL_PATH"], exist_ok=True)
    print(f"Created model weights path {params['MODEL_PATH']}")

if not os.path.exists(params["OUTPUT_PATH"]):
    os.makedirs(params["OUTPUT_PATH"], exist_ok=True)
    print(f"Created rollouts output path {params['OUTPUT_PATH']}")

## Modes

In [ ]:
def run_train(model, metadata, timestamp, params):
    os.makedirs(params["MODEL_PATH"], exist_ok=True)
    window_length = params["VELOCITY_CONTEXT_SIZE"] + 2
    train_ds = utils.io.load_dataset(
        params["DATA_PATH"],
        params["BATCH_SIZE"],
        split="train",
        mode="one_step_train",
        window_length=window_length 
    )
    valid_ds = utils.io.load_dataset(
        params["DATA_PATH"],
        params["BATCH_SIZE"],
        split="valid",
        mode="one_step",
        window_length=window_length
    )
    test_ds = utils.io.load_dataset(
        params["DATA_PATH"],
        params["BATCH_SIZE"],
        split="test",
        mode="one_step",
        window_length=window_length
    )
    try:
        for dummy in train_ds.take(1):
            model(dummy)  # Build
        checkpoint = utils.io.get_latest_checkpoint(params["MODEL_PATH"])
        model.load_weights(checkpoint)
    except FileNotFoundError:
        print("No saved model weights. Training from scratch.")
    try:
        steps_per_epoch = 100  # tunable
        model.fit(
            train_ds,
            epochs=params["NUM_STEPS"] // steps_per_epoch,
            steps_per_epoch=steps_per_epoch,  # save frequency
            validation_data=valid_ds,
            validation_freq=1,
            callbacks=[
                tf.keras.callbacks.ModelCheckpoint(
                    filepath=os.path.join(params["MODEL_PATH"], f"{timestamp}.weights.h5"),
                    save_weights_only=True,
                    save_best_only=True,
                    save_freq="epoch"
                ),
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=10,  # tunable
                    restore_best_weights=True,
                    verbose=1
                ),
                tf.keras.callbacks.LambdaCallback(
                    on_epoch_end=lambda epoch, logs: (
                        print(f"lr: {tf.keras.backend.get_value(model.optimizer.learning_rate):.4g}")
                    )
                )
            ]
        )
        metrics = model.evaluate(test_ds, return_dict=True, verbose=1)
        logging.info("Evaluation metrics:")
        for k, v in metrics.items():
            logging.info(f"{k}: {v:.6f}")


    except Exception as e:
        print(f"Exception occurred: {e}")
        model.save_weights(os.path.join(params["MODEL_PATH"], f"{timestamp}.crash.weights.h5"))
        print(f"Weights saved to {params['MODEL_PATH']}.")
        return

def run_eval(model, metadata, timestamp, params):
    eval_ds = utils.io.load_dataset(
        params["DATA_PATH"],
        params["BATCH_SIZE"],
        split=params["EVAL_SPLIT"],
        mode="one_step",
        window_length=params["VELOCITY_CONTEXT_SIZE"] + 2
    )
    for dummy in eval_ds.take(1):
        model(dummy)  # Build
    checkpoint = utils.io.get_latest_checkpoint(params["MODEL_PATH"])
    model.load_weights(checkpoint)
    metrics = model.evaluate(eval_ds, steps=1, return_dict=True)
    logging.info("Evaluation metrics:")
    for k, v in metrics.items():
        logging.info(f"{k}: {v:.6f}")

def run_rollout(model, metadata, timestamp, params):
    os.makedirs(params["OUTPUT_PATH"], exist_ok=True)
    rollout_ds = utils.io.load_dataset(
        params["DATA_PATH"],
        split=params["EVAL_SPLIT"],
        mode="rollout"
    )
    for dummy in rollout_ds.take(1):
        model(dummy)  # Build
    checkpoint = utils.io.get_latest_checkpoint(params["MODEL_PATH"])
    model.load_weights(checkpoint)
    num_steps =  metadata["sequence_length"] - params["VELOCITY_CONTEXT_SIZE"]
    for i, example in enumerate(rollout_ds.take(2), start=1):
        result = model.rollout(example, num_steps=num_steps)
        result["metadata"] = metadata
        filename = os.path.join(params["OUTPUT_PATH"], f"rollout_{params['EVAL_SPLIT']}_{i}.pkl")
        logging.info(f"Rollout {i} computed for {num_steps} steps. Saving to {filename}")
        with open(filename, "wb") as f:
            pickle.dump(result, f)

## Run simulation

In [6]:
def main(argv):
    metadata = utils.io.load_metadata(argv["DATA_PATH"])

    model = LearnedSimulator(
        dim=metadata["dim"],
        cutoff_radius=metadata["default_connectivity_radius"],
        boundaries=metadata["bounds"],
        noise_std=argv["NOISE_STD"],
        normalization_stats=utils.io.get_normalization_stats(metadata, argv["NOISE_STD"], argv["NOISE_STD"]),
        num_particle_types=argv["NUM_PARTICLE_TYPES"],
        static_particle_type_id=argv["STATIC_PARTICLE_ID"],
        velocity_context_size=argv["VELOCITY_CONTEXT_SIZE"]
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(  # S-G, p. 12
                initial_learning_rate=1e-4,
                decay_steps=argv["NUM_STEPS"],
                decay_rate=1e-2
            )  # 1e4 -> 1e6 exponentially over all training steps, can be more aggressive
        )
    )

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    if argv["MODE"] == "train":
        run_train(model, metadata, timestamp, argv)
    elif argv["MODE"] == "eval":
        run_eval(model, metadata, timestamp, argv)
    elif argv["MODE"] == "rollout":
        run_rollout(model, metadata, timestamp, argv)

In [ ]:
if __name__ == "__main__":
    # tf.get_logger().setLevel(logging.ERROR)  # Suppress tf warnings
    tf.config.run_functions_eagerly(True)
    if settings.TF_DEBUG_MODE:
        tf.data.experimental.enable_debug_mode()

    main(params)

/Users/dinbergare/anaconda3/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2025-06-09 14:05:10.300941: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


No saved model weights. Training from scratch.
Epoch 1/200000


2025-06-09 14:05:12.078618: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


100/100 [==============================] - ETA: 0s - loss: 6.2116

...

 lr: 9.998e-05
100/100 [==============================] - 271s 3s/step - loss: 0.3213 - val_loss: 6.1194e-09
Epoch 8/200000
100/100 [==============================] - ETA: 0s - loss: 0.2946

2025-06-09 14:40:01.581511: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


 lr: 9.998e-05
100/100 [==============================] - 272s 3s/step - loss: 0.2945 - val_loss: 1.1373e-08
Epoch 9/200000
100/100 [==============================] - ETA: 0s - loss: 0.2276

2025-06-09 14:44:52.649404: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


 lr: 9.998e-05
100/100 [==============================] - 292s 3s/step - loss: 0.2275 - val_loss: 5.5668e-09
Epoch 10/200000
 70/100 [====================>.........] - ETA: 52s - loss: 0.2285